# Analýza kandidátů do Poslanecké sněmovny napříč volbami
## Úvod
Tato práce se zabývá analýzou dat o kandidátech do Poslanecké sněmovny od roku 2006 do roku 2021. Zdrojem jsou [data z Českého statistického úřadu](https://volby.cz/opendata/opendata.htm).
* Analýza dat se snaží zodpovědět následující otázky:
    *  Jaké je věkové rozložení kandidátů napříč volbami?
    *  Jak velké zastoupení mají muži a ženy?
    *  Jaká ideologie převládá?
    *  Jaké obory jsou u kandidátů nejčastější?
    *  Jaký vliv má vysokoškolské vzdělání na volbu ideologie kandidátem?

Další součástí této práce je natrénování binárního klasifikačního modelu, kterým se na základě informací o kandydátech obsažených v datových zdrojích predikuje, zda kandidát ve volbých získá mandát a nebo nikoliv.

In [ ]:
# Installations
# !pip install matplotlib
# !pip install scikit-learn
# !pip install seaborn

## Import knihoven a inicializace potřebných funkcí
Ke splnění cílů této práce autoři užávají celkem pět knihoven jazyku Python, které jsou v dalším kroku práce naimportovány. Knihovny pandas a numpy se užívají k manipulaci s daty, matplotlib a seaborn pro vizualizaci a sci-kit learn pro účely strojového učení. Knihovna warnings je použita za účelem skrytí upozornění buněk.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# filtrace upozornění
warnings.filterwarnings("ignore")
# nastavení standardního rozměru grafu
plt.rcParams["figure.figsize"] = (15, 9)

Následující buňka zavádí funkci "obor". Cílem je povolání kandidátů (uvedené ve zdrojových datech) namapovat na širší/obecnější obory či kategorie (i.e. finance, právo, politika, řemeslo). Kandidáti totiž mouhou mít ve sloupci POVOLANI uvedeno cokoliv a proto bylo nutné vytvořit předem definovaný slovník, jehož pomocí se pomocí hodnoty sloupce POVOLANI, každému kandidátovi přiřadí obecnější kategorie, do níž jeho povolání spadá. K tomu autoři vytvořili JSON soubor (viz dict_occupations.json). Ten jako hodnoty vnořených klíčů "var" uvádí různé varianty názvu povolání, ke kterým se má mapovat příslušná kategorie. Kategorie byly utvořeny a definovány na základě nejčastějších hodnot ve sloupci POVOLANI. Pokud k dané hodnotě ve sloupci POVOLANÍ neexistuje kategorie, pak se přiřadí kategorie "Jiné".

In [ ]:
import json # daná knihovna umožňuje snadný import JSON souborů ve formátu python slovníků

with open("dict_occupations.json", "r", encoding="utf-8") as file: # otevření JSON souboru
    occup_dict = json.load(file)


def obor(val: str) -> str:
    """
    Checks to what field does candidate's profession belongs to (such as finance, law, medicine etc.) with the help of keywords from global occup_dict dictionary
    :param val: string value containing candidate's profession description
    :return: string value with the particular field, returns "Jiné" if no matches were found
    """
    global occup_dict
    for key in occup_dict.keys():
        for i in occup_dict[key]["var"]:
            if i in val.lower():
                return occup_dict[key]["name"]
    return "jiné"


occup_dict.keys()

Jelikož jsou použita volební data z parlamentních voleb v různých letech, které mají více či méně standardizovaný tvar, autoři užívají jednu předem definovanou funkci na čištění všech datových souborů.

In [ ]:
def clean_data(data: pd.DataFrame, strany: dict) -> pd.DataFrame:
    """
    Clean and prepare a dataframe containing data from different types of elections for further visualisation and modelling purposes.
    The output is:
        -[cele_jmeno]: combining name columns into one
        -[vek]: candidate's age
        -[vs_vzdelani]: Creating a boolean column checking whether a candidate has university degree
        -[obor]: field of candidate's profession
        -[pohlavi]: candidate's gender (based on their surname's ending)
        -[kraj]: region where candidate runs their campaign
        -[NSTRANA]: political party that nominated the candidate
        -[ideologie]: canidate's ideology (left-wing, center, right-wing)
        -[mandat]: whether candidate got their mandate
        -[year]: year of elections
    :param data: pandas dataframe including elections data
    :param strany: dictionary data type with political parties' data
    :return: cleaned pandas dataframe
    """
    df = data.copy()

    df["cele_jmeno"] = df["JMENO"] + " " + df["PRIJMENI"]
    df["pohlavi"] = np.where(df.loc[:, "cele_jmeno"].str[-1] == "á", "F", "M")
    df["vs_vzdelani"] = np.where((df["TITULPRED"].notnull()) | (df["TITULZA"].notnull()), 1, 0)
    df["vek"] = df["VEK"]

    # seznam krajů dle kódů
    kraj_list = ["Hlavní město Praha", "Středočeský kraj", "Jihočeský kraj", "Plzeňský kraj", "Karlovarský kraj",
                "Ústecký kraj", "Liberecký kraj", "Královéhradecký kraj", "Pardubický kraj", "Kraj Vysočina",
                "Jihomoravský kraj", "Olomoucký kraj", "Zlínský kraj", "Moravskoslezský kraj"]
    kraj_dict = dict(zip(list(range(1, 15)), kraj_list))
    df["ideologie"] = df["NSTRANA"].map(strany)
    df["kraj"] = df["VOLKRAJ"].map(kraj_dict)
    df["obor"] = df["POVOLANI"].apply(lambda row: obor(str(row))) # mapuje slovník oborů na sloupec "POVOLANI"
    df["mandat"] = df["MANDAT"].apply(lambda row: 1 if str(row) == "A" or str(row) == "1" else 0) # převádí různé záznamy na jeden tvar (v některých letech je to A/N, někde 1/0)
    columns = ["cele_jmeno", "vek", "vs_vzdelani", "obor", "pohlavi", "kraj", "NSTRANA", "ideologie", "mandat", "year"]

    return df[columns]

## Načítání dat
Popis některých sloupců, které jsou součástí výstupního souboru:
   * **VOLKRAJ** - číslo kraje, na základě těchto čísel se navázují názvy krajů
   * **JMENO**, **PRIJMENI** - jméno a příjmení, na základě koncovky u příjmení se vytváří sloupec "pohlavi"
   * **TITULPRED**, **TITULZA** - tituly před a za jménem, při přitomnosti aspoň jedné ze dvou se sloupci "vs_vzdelani" přiřazuje hodnota 1, jinak 0
   * **VEK** - věk
   * **POVOLANI** - povolaní kandidáta, jak bylo uvedeno výše - tuto hodnotu kandidáti vyplňují sámi a proto jeho honoty nemají žádný standard. Sloupec "obor" ve výstupním souboru vzniká mapováním sloupce POVOLANI pomocí slovníku dict_occupations.json.
   * **NSTRANA** - strana, která kandidáta navrhla - ze všech sloupců se obahující strany byl použit tento, neboť obsahuje nejméně chybějících hodnot. Je tomu tak proto, že každého kandidáta musela nějaká strana navrhnout. Na rozdíl od např. KSTRANA, která označuje stranu kandidáta, kde se může vyskytovat kandidát bez strany. Hlavním účelem tohoto sloupce je uržení tzv. **ideologie**, pomocí ní dochází ke třídění kandidátů do úzších skupin dle jednodimenzionálního rozdělení (levicové, středové a pravicové strany).
   * **MANDAT** - zda kandidát nakonec získal mandát nebo nikoliv
   * **year** - rok voleb - přídan během načítání dat

In [ ]:
# data budou staženy do listu snem_list
snem_list = []
snem_years = [2006, 2010, 2013, 2017, 2021] # jednotlivé roky voleb
for year in snem_years:
    temp_df = pd.read_csv(f"legislative-elections/snem_{year}.csv",
                          sep=";", encoding="utf-8")
    temp_df["year"] = year
    snem_list.append(temp_df)

snem_list[0].head(10)

Následující dataframe obsahuje data o stranách, které se zúčastnily voleb do sněmovny v letech 2006 až 2021. Sloupec **Ideologie** je přidán ručně na základě veřejně dostupných informací ohledně každé strany (jednodimenzionální rozdělení na levicové, středové a pravicové strany).

In [ ]:
strany = pd.read_csv("legislative-elections/strany.csv",
                    sep=";", encoding="utf-8")
strany_dict = dict(zip(strany["VSTRANA"], strany["Ideologie"]))
strany.head()

Sloupec "pohlavi" je vytvořen na základě koncovky příjmení kandidáta.

In [ ]:
# Čištění dat pomocí funkce clean_data
snem_clean_list = []

for i in range(5):
    temp_df = clean_data(snem_list[i], strany_dict)
    snem_clean_list.append(temp_df)

snem_clean_list[0].head(10)

In [ ]:
# Tato funkce dává v dané buňce veškeré dataframy do jednoho, jelikož pro budoucí účely je nutné mít data ve formátu long
snem_long = pd.concat(snem_clean_list, ignore_index=True)
snem_long.tail(10)

## Ošetření chybějících hodnot
Nyní je nutné zkontrolovat, zda nějaké data nechybí.

In [ ]:
print("Počet chybějících hodnot v každém sloupci:\n")
print(snem_long.isnull().sum())
print("-" * 40)
print("Nejčastější ideologie kandidátů:\n")
print(snem_long["ideologie"].value_counts())

Jak je zřejmé z výstupu nad, chybějící hodnoty jsou ve sloupcích vek, NSTRANA a ideologie.
Nejčastější ideologie je pravicová. Tedy lze bez ztráty informace zaměnit NA hodnoty za hodnotu **pravicová**.
NA hodnoty ve sloupcích vek a NSTRANA patří neexistujícím kandidátům, tudíž je lze odstranit.

In [ ]:
snem_long[snem_long["vek"].isnull()]

In [ ]:
# Veškerým NA hodnotám ve sloupci ideologie budou přiřazeny "pravicová" ideologie, jelikož je nejčastější

snem_long = snem_long.dropna(subset=["vek"])
snem_long.ideologie = snem_long.ideologie.fillna("Pravice")

print("Stav po ošetření chybějících hodnot:")
print("Počet chybějících hodnot v každém sloupci:\n")
print(snem_long.isnull().sum())
print("-" * 40)
print("Nejčastější ideologie kandidátů:\n")
print(snem_long["ideologie"].value_counts())

## Exploratorní analýza
Další částí této práce je exploratorní datová analýza.
První řadě je nutné zkontrolovat datové typy jednotlivých sloupců:

In [ ]:
snem_long.info()

Všechny kategoriální sloupce převedeme na příslušný datový typ, proměnná věk bude integerem.
Tímto vzniká situace, kdy je většina proměnných kategoriální, což napomáhá zvýšení početu dimenzí pro trénování modelů.

In [ ]:
for col in ["vs_vzdelani", "obor", "pohlavi", "kraj", "NSTRANA", "ideologie", "mandat", "year"]:
    snem_long[col] = snem_long[col].astype("category")

snem_long["vek"] = snem_long["vek"].astype("int64")

snem_long.info()

První vizualizací je rozložení věků kandidátů podle pohlaví v jednotlivých volebách. Každý graf vyobrazuje míry centrální tendence.

In [ ]:
# knihovna seaborn a distribution plot, samostatný graf pro každý volební rok
g = sns.displot(
        data=snem_long, x="vek", hue="pohlavi", kind="kde",
        fill=True, legend=True, col="year", col_wrap=3, hue_order=["M", "F"],
        height=5
    )
g.set_titles("Rozložení věků kanidátů do Poslanecké sněmovny\n podle pohlaví v roce {col_name}",
             size="large")
g.set(xlabel="Věk kandidáta", ylabel="Hustota")
sns.move_legend(g, "center right", bbox_to_anchor=(.75, .45), frameon=True, fontsize="medium",
                title="Pohlaví", title_fontsize="medium", labels=["Ženy", "Muži"])

# svislé přímky pro označení míry centrální tendence
years_avgs = {}
for i, year in enumerate(snem_years):
    years_avgs[i] = [snem_long.query(f"year == {year}")["vek"].mean(),
                     snem_long.query(f"year == {year}")["vek"].median(),
                     snem_long.query(f"year == {year}")["vek"].mode().iat[0]]

# přímky pro jednotlivé grafy rozložení
axes = g.axes.flatten()
for i, ax in enumerate(axes):
    ax.axvline(years_avgs[i][0], ls="--", lw=1.0, label=f"""Průměr: {years_avgs[i][0]:.2f}""")
    ax.axvline(years_avgs[i][1], color="darkred", lw=1.0, label=f"""Medián: {years_avgs[i][1]:.2f}""")
    ax.axvline(years_avgs[i][2], ls="-.", color="orange", lw=1.0, label=f"""Modus: {years_avgs[i][2]:.2f}""")
    ax.legend(loc=0)

Podle výstupu lze usoudit, že poměr mužů a žen je u většiny věkových skupin 2-2,5 ku 1. Mužů tudíž kandiduje víc.
Rozdělení jsou špičatá, ale symetrická kolem 47 let, jak to ukazují průměry a mediány. Lze usoudit, že u voleb do poslanecké sněmovny nedochází k nerovnoměrnému rozdělení věků a žádná věková skupina nepřevládá.

Nyní zhodnoťme míry centrální tendence věku pro každou ideologii zvlášť:

In [ ]:
snem_long.groupby("ideologie").vek.describe()

Z tabulky je zřejmé, že je každá skupina zastupena rovnoměrně a příliš se od ostatních neliší.

Druhým grafem znázorňuje porovnání ideologií kandidátů podle krajů u voleb v roce 2021 a také jaké zastoupení mají zvolené ideologické kategorie ve sněmovně:

In [ ]:
# pořadí tříd a příslušných barev
hue_order = ["Levice", "Střed", "Pravice"]
hue_palette = ["Red", "Green", "Blue"]

# první graf
sns.countplot(snem_clean_list[4], y="kraj", hue="ideologie",
              hue_order=hue_order, palette=hue_palette)
plt.title("Ideologie kandidátů do sněmovny podle krajů v roce 2021")
plt.xlabel("Počet")
plt.ylabel("Kraj")
plt.legend(title="Ideologie")
plt.show()

# druhý graf
sns.countplot(snem_clean_list[4][snem_clean_list[4]["mandat"] == 1], y="kraj", hue="ideologie",
              hue_order=hue_order, palette=hue_palette)
plt.title("Ideologie zvolených kandidátů do sněmovny podle krajů v roce 2021")
plt.xlabel("Počet")
plt.ylabel("Kraj")
plt.legend(title="Ideologie")
plt.show();

Lze vidět, že pravicové ideologie převládají u kandidátů ve všech krajích, ale do sněmovny se nakonec dostávají nejčastěji kandidáti ze středu.

Třetí vizualizací představuje zastoupení různých oborů podle VŠ vzdělání.

In [ ]:
plt.figure(figsize=(15, 9))
g = sns.countplot(
    data=snem_long, x="obor", hue="vs_vzdelani",
    order=snem_long.obor.value_counts().index
)
plt.title("Nejčastější povolání podle vysokoškolského vzdělání v rocích 2006 - 2021")
plt.xlabel("Povolání")
plt.xticks(rotation=90)
plt.ylabel("Počet")
plt.legend(title="Vysokoškolské vzdělání",
           labels=["Ne", "Ano"]);

Nejčastěji jdou do voleb lidé ze soukromého sektoru bez vysokoškolského vzdělání, druhou nejčastější skupinou jsou, kteří již v politice působí (např. starostové, hejtmani), kteří většinou za sebou vysokou školu mají, třetí nejčastější skupinu kandidátů zastupují podnikatelé.

Následný graf ukazuje, jaký vliv má vzdělání na ideologickou kategorii jíž kandidát reprezentuje:

In [ ]:
sns.catplot(
    data=snem_long, x="vs_vzdelani", y="vek",
    hue="ideologie", hue_order=hue_order, palette=hue_palette,
    height = 8, aspect = 1.25
)
plt.title("Ideologie kandidátů do Poslanecké sněmovny podle vysokoškolského vzdělání")
plt.xlabel("Vysokoškolské vzdělání")
plt.ylabel("Věk kandidáta");

Kandidáti bez vysokoškolského vzdělání nejčastěji reprezentují pravicovou ideologii, zatímco kandidáti se vzděláním jsou spíše středo-levicoví. Avšak podíl levicových kandidátů je značný jen u starších věkových skupin.

## Využití klasifikačního modelu za účelem predikce zvolení kandidáta
Nejdříve je nutné provést tzv. dummifying kategoriálních proměnných, tj. převedou se na vícedimenzionální tvar

In [ ]:
# Dummifying kategoriálních proměnných
snem_ml = snem_long.drop(["cele_jmeno", "NSTRANA", "year"], axis=1) # tyto proměnné nejsou potřeba, jelikož nemají velký vliv na cílovou proměnnou a mají hodně dimenzí
snem_ml = pd.get_dummies(data=snem_ml, columns=["vs_vzdelani", "obor", "pohlavi", "kraj", "ideologie"])
snem_ml.head()

Jak je zřejmé z grafu níže, cílová proměnná "mandat" je silně nebalancovaná - na mandát dosáhlo jen 1000 kandidátů ze skoro 29 tisíc.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(
    data=snem_ml, x="mandat"
)
print(snem_ml["mandat"].value_counts())

In [ ]:
from sklearn.svm import SVC # Support Vector Machine Classifier algorithm
from sklearn.ensemble import RandomForestClassifier # Random Forest Classifier
from sklearn.preprocessing import StandardScaler # škálování dat
from sklearn.decomposition import PCA # Principal Component Analysis

Metoda undersampling byla zvolena s cílem zabránit nevyváženosti dat, tj. náhodně se zvolí stejný počet kandidátů bez mandátu jako s mandátem.

In [ ]:
from sklearn.utils import resample

sn_maj = snem_ml[snem_ml["mandat"] == 0] # rozdělení datasetu na minor/major
sn_min = snem_ml[snem_ml["mandat"] == 1]

sn_maj_und = resample(sn_maj,
                     replace=True,
                     n_samples=1000, # vyrovnání se s minoritní třídou
                     random_state=42)

# sjednocení obou setů - tím se vyváží
snem_und = pd.concat([sn_maj_und, sn_min])
print(snem_und["mandat"].value_counts())

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(
    data=snem_und, x="mandat"
);

Nyní je cílová proměnná vyvážená.

Dalším krokem je rozdělení na testovací a trénovací datasety.

In [ ]:
from sklearn.model_selection import train_test_split
# dělení sloupců na prediktory a cílovou proměnnou
predictors = snem_ml.columns.to_list()
predictors.remove("mandat")
target = ["mandat"]

X = snem_und[predictors]
y = snem_und[target]

# dělení dat na trénovací a testovací sety
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"Rozměr trénovacího datasetu: {len(X_train)}")
print(f"Rozměr testovacího datasetu: {len(X_test)}")

Pro snadnou práci s více modely najednou jsou použity pipeliny. Porovnání Support Vector Classifier bez preprocessingu a Random Forest Classifier s PCA - PCA nastaveno s 95% objasněním variability komponentů.

In [ ]:
from sklearn.pipeline import Pipeline
# vytvoření pipeliny jen s SVC modelem
svc_pipeline = Pipeline([("svc", SVC(class_weight="balanced", random_state=42))]) # Support Vector Machine Classifier

# pipeline se škalováním (potřebné pro PCA), PCA a samotným Random Forest modelem
randf_pipeline = Pipeline([("scaler", StandardScaler()),
                         ("pca", PCA(n_components=0.95)),
                         ("randf", RandomForestClassifier(class_weight="balanced", random_state=42))]) # Random Forest Classifier with PCA decomposition

# mřížky hyperparametrů
svc_grid = {"svc__kernel": ["linear", "rbf"],
        "svc__C": [0.1, 1, 10, 100]}

randf_grid = {"randf__n_estimators": [200, 400, 600],
              "randf__max_depth": [10, 50, 100],
              "randf__min_samples_split": [2, 5, 10]}

* **GridSearch** hledá následující hyperparametry:
    * **SVC**: kernel a C. Prvním parametrem je jádro daného modelu - v našem případě jsou to lineární a rbf. Druhým parametrem je síla regularizace: 0.1, 1, 10 a 100.
    * **Random Forest**: n_estimators, max_depth a min_sample_split. Prvním je počet stromů ve forestu. Druhým - maximální hloubka jednotlivých stromů. Třetí parametr reguluje minimální počet vzorků v jednom stromu.

Nejlepší hyperparametry jsou vybrány dle metriky ROC AUC (Plocha pod ROC křivkou). V tomto případě obyčejná přesnost může být nepostačující z důvodu nevyváženosti originálního datasetu. Je nutné zajistit, aby True Positive Rate byl větší proto, aby klasifikátor správně přiřazoval pozitivní hodnoty.

In [ ]:
from sklearn.model_selection import GridSearchCV

results = {}
pipelines = [svc_pipeline, randf_pipeline]
grids = [svc_grid, randf_grid]
names = ["SVC", "Random Forest"]

# aplikování gridsearch na jednotlivé modely a mřížky
for name, pipe, grid in zip(names, pipelines, grids):
    print(f"{name} se zpracovává\n")
    temp_search = GridSearchCV(pipe, grid,
                               cv=3, n_jobs=3, # počet Cross Validations je jen 3 pro lepší výkonnost
                               scoring="roc_auc")
    temp_search.fit(X_train, y_train.values.ravel())
    results[name] = {}
    results[name]["model"] = temp_search.best_estimator_
    results[name]["params"] = temp_search.best_params_ # uložení nejlepších parametrů

print("Proces se skončil úspěšně\n")
for name in names:
    print(f"""Nejlepší parametry {name} modelu jsou {results[name]["params"]}\n""")

In [ ]:
from sklearn.metrics import classification_report
class_names = ["Mandat 0", "Mandat 1"]
for name in names:
    print(f"Výsledky {name} modelu na testovacím datasetu:")
    print(classification_report(y_test.values.ravel(),
                                results[name]["model"].predict(X_test),
                                target_names=class_names))
    print("-" * 60)

Přesnost obou algoritmů je skoro stejná: 0.78 u SVC a 0.76 u Random Forest modelu. Nyní zobrazme matice záměn každého modelu:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
_, ax = plt.subplots(figsize=(7, 7))
svc_pred = results["SVC"]["model"].predict(X_test) # uložení predikcí modelu
svc_cm = confusion_matrix(y_test, svc_pred) # vytvoření matici záměn
svc_confm = ConfusionMatrixDisplay(confusion_matrix=svc_cm,
                                   display_labels=class_names) # vizualizace této matice
svc_confm.plot(ax=ax, cmap=plt.cm.Blues);

In [ ]:
_, ax = plt.subplots(figsize=(7, 7))
randf_pred = results["Random Forest"]["model"].predict(X_test)
randf_cm = confusion_matrix(y_test, randf_pred)
randf_confm = ConfusionMatrixDisplay(confusion_matrix=randf_cm,
                                   display_labels=class_names)
randf_confm.plot(ax=ax, cmap=plt.cm.Blues);

Jak lze vidět, matice jsou si velice podobné. Nicméně True Positives Rate je větší u SVC modelu, než u Random Forest, což je pro naši klasifikaci lepší (chceme menší počet False Negatives). Přesto je rozdíl víceméně zanedbatelný.

In [ ]:
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20, 6))

svc_fpr, svc_tpr, _ = roc_curve(y_test, svc_pred) # kalkulace False Positive Rate a True Positive Rate pro nakreslení ROC křivky
svc_auc = auc(svc_fpr, svc_tpr) # kalkulace plochy pod křivkou ROC
svc_roc = RocCurveDisplay(fpr=svc_fpr, tpr=svc_tpr, roc_auc=svc_auc) # vizualizace samotné křivky
svc_roc.plot(ax=ax1, name="SVC")

randf_fpr, randf_tpr, _ = roc_curve(y_test, randf_pred)
randf_auc = auc(randf_fpr, randf_tpr)
randf_roc = RocCurveDisplay(fpr=randf_fpr, tpr=randf_tpr, roc_auc=randf_auc)
randf_roc.plot(ax=ax2, name="Random Forest")
plt.show();

Plochy pod křivkou u obou modelů jsou si opět podobné. SVC dosáhl 0.78, Random Forest - 0.76.
Finálně můžeme tedy usoudit, že na daném datasetu dosahuje SVC lepších výsledků oproti Random Forest klasifikátoru, i když byli pro Random Forest uplatněny preprocessingové kroky.

## Závěr
* Při zpracování této práce byly zjištěny následující výsledky:
    * věkové rozložení kandidátů je symetrické, žádná věková skupina nepřevládá,
    * poměr muži:ženy je 2-2,5 ku 1 ve většině věkových skupin,
    * pravicová ideologie převládá u kandidátů, ale nakonec největší počet zvolených je ze středu,
    * nejčastěji do Poslanecké sněmovny kandidují zaměstnanci ze soukromého sektoru, podnikatelé a stávající politici.
    * kandidáti bez vysokoškolského vzdělání mají větší tendenci být pravicovými, než-li kandidáti se vysokoškolským stupněm vzdělání - ti jsou většinou ve středu politického spektra.

Pro trénování klasifikačního modelu bylo použito dvou metodik: **Support Vector Machine Classifier** bez preprocessingu a **Random Forest Classifier** s PCA preprocessingem. Originální dataset je velice nevyvažený, proto bylo použito **undersamplingu**. Hyperparametry obou modelů byly zvoleny pomocí **Grid Search** algoritmu metrikou **AUC**. Ve výsledku měly oba modely skoro shodné výsledky - **SVC** model dosáhl o 0,02 větší plochy pod křivkou a **F-1** skóre -- **0.78**.